# Segundo Parcial Práctica

In [ ]:
#r "nuget: Microsoft.DotNet.Interactive.SqlServer, *-*"

In [ ]:
foreach (var line in System.IO.File.ReadLines(".env"))
{
    var parts = line.Split('=');
    if (parts.Length == 2)
    {
        var key = parts[0].Trim();
        var value = parts[1].Trim();
        Environment.SetEnvironmentVariable(key, value);
    }
}

string connectionString = "Server=" + Environment.GetEnvironmentVariable("Server") + ";Database=" + Environment.GetEnvironmentVariable("Database") + ";User Id=" + Environment.GetEnvironmentVariable("User") + ";Password=" + Environment.GetEnvironmentVariable("Password") + ";Encrypt=True;TrustServerCertificate=True;Connection Timeout=30;";


In [ ]:
#!connect mssql --kernel-name Pampero @csharp:connectionString

1)	Listar los clientes que no hayan realizado pedidos

a) Con Subconsulta

b) Con Outer JOIN

In [ ]:
SELECT IDCliente, NombreEmpresa
FROM Clientes
WHERE IDCliente NOT IN
(SELECT IDCliente 
FROM Pedidos
WHERE IDCliente IS NOT NULL
);

SELECT IDCliente, NombreEmpresa
FROM Clientes
WHERE NOT EXISTS
(SELECT 1
FROM Pedidos P
WHERE P.IDCliente = Clientes.IDCliente
);

In [ ]:
SELECT C.IDCliente, NombreEmpresa
FROM Clientes C
LEFT JOIN Pedidos P ON P.IDCliente = C.IDCliente
WHERE P.IDCliente IS NULL;

2)	Listar los clientes que no hayan realizado pedidos de productos de la categoría “Mariscos”

a) Con Subconsulta

b) Con Outer JOIN

In [ ]:
SELECT IDCliente, NombreEmpresa
FROM Clientes
WHERE IDCliente NOT IN
(SELECT IDCliente 
FROM Pedidos P JOIN [Detalles Pedido] DP ON DP.IDPedido = P.IDPedido
JOIN Productos PR ON PR.IDProducto = DP.IDProducto
JOIN Categorias C ON C.IDCategoria = PR.IDCategoria
WHERE C.nombrecategoria = 'Mariscos' AND P.IDCliente IS NOT NULL
);

SELECT C.IDCliente, NombreEmpresa
FROM Clientes C
WHERE NOT EXISTS
(SELECT 1
FROM Pedidos P JOIN [Detalles Pedido] DP ON DP.IDPedido = P.IDPedido
JOIN Productos PR ON PR.IDProducto = DP.IDProducto
JOIN Categorias CA ON CA.IDCategoria = PR.IDCategoria
WHERE CA.nombrecategoria = 'Mariscos' AND P.IDCliente = C.IDCliente
);

In [ ]:
SELECT C.IDCliente, NombreEmpresa
FROM Clientes C
LEFT JOIN (Pedidos P
JOIN [Detalles Pedido] DP ON DP.IDPedido = P.IDPedido
JOIN Productos PR ON PR.IDProducto = DP.IDProducto
JOIN Categorias CA ON CA.IDCategoria = PR.IDCategoria AND CA.nombrecategoria = 'Mariscos') ON C.IDCliente = P.IDCliente
WHERE P.IDPedido IS NULL;

SELECT C.IDCliente, NombreEmpresa
FROM Pedidos P
JOIN [Detalles Pedido] DP ON DP.IDPedido = P.IDPedido
JOIN Productos PR ON PR.IDProducto = DP.IDProducto
JOIN Categorias CA ON CA.IDCategoria = PR.IDCategoria AND CA.nombrecategoria = 'Mariscos'
RIGHT JOIN Clientes C ON C.IDCliente = P.IDCliente
WHERE P.IDPedido IS NULL;

3)	Considerando el siguiente esquema de generaciones, agrupe y cuente los empleados por generación y puesto.
 
Tener en cuenta que lógica para determinar la generación debe escribirse una sola vez en la consulta.


In [ ]:
SELECT Generation, puesto, COUNT(*) AS Cantidad
FROM
(SELECT *, 
CASE 
WHEN YEAR(FechaNacimiento) < 1901 THEN 'Lost Generation'
WHEN YEAR(FechaNacimiento) < 1928 THEN 'Greatest Generation'
WHEN YEAR(FechaNacimiento) < 1946 THEN 'Silent Generation'
WHEN YEAR(FechaNacimiento) < 1965 THEN 'Baby Boomers'
WHEN YEAR(FechaNacimiento) < 1981 THEN 'Generation X'
WHEN YEAR(FechaNacimiento) < 1997 THEN 'Millennials/Generation Y'
WHEN YEAR(FechaNacimiento) < 2013 THEN 'Zoomers/Generation Z'
ELSE 'Generation Alpha'
END AS Generation
FROM Empleados) AS EG
GROUP BY Generation, puesto
ORDER BY Generation, puesto;

WITH Generations AS
(SELECT *, 
CASE 
WHEN YEAR(FechaNacimiento) < 1901 THEN 'Lost Generation'
WHEN YEAR(FechaNacimiento) < 1928 THEN 'Greatest Generation'
WHEN YEAR(FechaNacimiento) < 1946 THEN 'Silent Generation'
WHEN YEAR(FechaNacimiento) < 1965 THEN 'Baby Boomers'
WHEN YEAR(FechaNacimiento) < 1981 THEN 'Generation X'
WHEN YEAR(FechaNacimiento) < 1997 THEN 'Millennials/Generation Y'
WHEN YEAR(FechaNacimiento) < 2013 THEN 'Zoomers/Generation Z'
ELSE 'Generation Alpha'
END AS Generation
FROM Empleados)
SELECT Generation, puesto, COUNT(*) AS Cantidad
FROM Generations
GROUP BY Generation, puesto
ORDER BY Generation, puesto;

SELECT *,
CASE 
WHEN YEAR(FechaNacimiento) < 1901 THEN 'Lost Generation'
WHEN YEAR(FechaNacimiento) < 1928 THEN 'Greatest Generation'
WHEN YEAR(FechaNacimiento) < 1946 THEN 'Silent Generation'
WHEN YEAR(FechaNacimiento) < 1965 THEN 'Baby Boomers'
WHEN YEAR(FechaNacimiento) < 1981 THEN 'Generation X'
WHEN YEAR(FechaNacimiento) < 1997 THEN 'Millennials/Generation Y'
WHEN YEAR(FechaNacimiento) < 2013 THEN 'Zoomers/Generation Z'
ELSE 'Generation Alpha'
END AS Generation
INTO #EmpleadosGenerations
FROM Empleados;
SELECT Generation, puesto, COUNT(*) AS Cantidad
FROM #EmpleadosGenerations
GROUP BY Generation, puesto
ORDER BY Generation, puesto;